In [1]:
import uproot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import awkward as ak
import numpy as np
import math
import ROOT as root
from array import array

Welcome to JupyROOT 6.26/16


In [2]:
#Experiment with geometry loading
f = root.TFile.Open("/sdf/home/t/tanaka/MicroProdN4p1_NDComplex_FHC.spill.full.0002459.EDEPSIM_SPILLS.root")
f.ls()
geom = f.Get("EDepSimGeometry")

TFile**		/sdf/home/t/tanaka/MicroProdN4p1_NDComplex_FHC.spill.full.0002459.EDEPSIM_SPILLS.root	
 TFile*		/sdf/home/t/tanaka/MicroProdN4p1_NDComplex_FHC.spill.full.0002459.EDEPSIM_SPILLS.root	
  KEY: TTree	EDepSimEvents;26	Energy Deposition for Simulated Events [current cycle]
  KEY: TTree	EDepSimEvents;25	Energy Deposition for Simulated Events [backup cycle]
  KEY: TGeoManager	EDepSimGeometry;1	Simulated Detector Geometry
  KEY: TMap	event_spill_map;1	A (key,value) map
  KEY: TParameter<double>	spillPeriod_s;1	
  KEY: TParameter<double>	pot_per_spill;1	
  KEY: TParameter<double>	pot1;1	
  KEY: TParameter<double>	pot2;1	
  KEY: TDirectoryFile	DetSimPassThru;1	DetSimPassThru


Info in <TGeoManager::CloseGeometry>: Geometry loaded from file...
Info in <TGeoManager::SetTopVolume>: Top volume is volWorld_PV. Master volume is volWorld_PV
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CountLevels>: max level = 23, max placements = 1535
Info in <TGeoManager::CloseGeometry>: 1643903 nodes/ 8916 volume UID's in Simulated Detector Geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------


In [3]:
print(type(geom))           # Should show <class 'ROOT.TGeoManager'>
print(geom.GetName())       # Usually 'EDepSimGeometry' or similar
geom.Print()
root.gGeoManager = geom

<class cppyy.gbl.TGeoManager at 0x55d7ec750d30>
EDepSimGeometry
OBJ: TGeoManager	EDepSimGeometry	Simulated Detector Geometry


In [4]:
top_volume = geom.GetTopVolume()
top_volume.GetName()

'volWorld_PV'

In [5]:
top_volume = geom.GetTopVolume()
print(f"Top volume: {top_volume.GetName()}")
print(f"Number of daughter nodes: {top_volume.GetNdaughters()}")
#top_volume.GetNodes()

Top volume: volWorld_PV
Number of daughter nodes: 1


In [6]:
xval, yval, zval = 0, 0, 11190
node = geom.FindNode(xval, yval, zval)
if not node:
    print("No node found at this position.")
else:
    print("Node name:", node.GetName())

Node name: volDetEnclosure_PV_0


starting from the scintillator bar, moving up in the geometry to see the full layering
There are two rotations of bars, and their names are literally "modulelayervol1" - U Bar +3 from Y and "modulelayervol2" - V Bar -3 from Y"
"modulelayervol3" - X Bar, "modulelayervol4" - Theoretical pure Y bar...

In [7]:
#This is an example hit in the TMS to explore its location!
x, y, z = -2789.9808, -385.6931, 17431.5356
node = geom.FindNode(x, y, z) #this find the node the hit is in - a node is an instance of a volume. 

#print(node.GetVolume().GetName())
print(node.GetName()) #this one is in the 16th instance of the scinBoxlvTMS_PV volume. 

#moving up a level
geom.CdUp()
node = geom.GetCurrentNode()
print(node.GetName())

#moving up a level
geom.CdUp()
node = geom.GetCurrentNode()
print(node.GetName())

#moving up a level
geom.CdUp()
node = geom.GetCurrentNode()
print(node.GetName())

#moving up a level
geom.CdUp()
node = geom.GetCurrentNode()
print(node.GetName())

#moving up a level
geom.CdUp()
node = geom.GetCurrentNode()
print(node.GetName())

#moving up a level
geom.CdUp()
node = geom.GetCurrentNode()
print(node.GetName())

#once we hit volWorld_PV_1 we have escaped the detector

scinBoxlvTMS_PV_16
ModuleBoxvol_PV_0
modulelayervol1_PV_44
volTMS_PV_0
volDetEnclosure_PV_0
rockBox_lv_PV_0
volWorld_PV_1


As we can see, this hit is located in a U Bar! Let's see if we can automate this scanning process a bit:

In [14]:
x_hit, y_hit, z_hit = -2825.8306180555574, -447.8311449193576, 17594.793326825627 #-2789.9808, -385.6931, 17431.5356 

def ModuleFinder(x,y,z,geom):
    module_names = {'modulelayervol1' : 'U' ,'modulelayervol2' : 'V' ,'modulelayervol3': 'X' ,'modulelayervol4' : 'Y'}
    node = geom.FindNode(x, y, z)
    orientation = 'null'
    bar_number = 'null'
    layer_number = 'null'
    while node:
        #print(node.GetName())
        #print(node.GetNumber())
        if 'scinBox' in node.GetName(): #this is like a rlly stupid line but let's just hope it works
            bar_number = node.GetNumber()
            box = geom.GetCurrentVolume().GetShape()
            xw = 2 * box.GetDX()
            yw = 2 * box.GetDY()
            zw = 2 * box.GetDZ()
        if "modulelayervol" in node.GetName():
            layer_number = node.GetNumber()
            for module_name in module_names.keys():
                if module_name in node.GetName():
                    orientation = module_names[module_name]
                    #print(module_name)
        if 'volDetEnclosure' in node.GetName():
            break
        geom.CdUp()
        node = geom.GetCurrentNode()

    return (orientation, bar_number, layer_number)

bar_orientation, bar_no, layer_no = ModuleFinder( x_hit, y_hit, z_hit, geom) 
print(bar_orientation)
print(bar_no)
print(layer_no)

U
15
45


In [ ]:
Bang